In [1]:
import requests
import json
import csv
import pandas as pd
import matplotlib as plt
import numpy as np
from config import api_key

In [3]:
csvfile = "./Resources/2019.csv"
csv_df = pd.read_csv(csvfile)
csv_df["Year"] = "2019"
len(csv_df)

620

In [4]:
csvfile18 = "./Resources/2018.csv"
csv_df18 = pd.read_csv(csvfile18)
csv_df18["Year"] = "2018"
len(csv_df18)

622

In [5]:
csvfile17 = "./Resources/2017.csv"
csv_df17 = pd.read_csv(csvfile17)
csv_df17["Year"] = "2017"
len(csv_df17)

571

In [10]:
csvfile16 = "./Resources/2016.csv"
csv_df16 = pd.read_csv(csvfile16)
csv_df16["Year"] = "2016"
len(csv_df16)

584

In [12]:
csvfile15 = "./Resources/2015.csv"
csv_df15 = pd.read_csv(csvfile15)
csv_df15["Year"] = "2015"
len(csv_df15)

595

In [64]:
combined_df = csv_df.append(csv_df18)
combined_df = combined_df.append(csv_df17)
combined_df = combined_df.append(csv_df16)
combined_df = combined_df.append(csv_df15)
combined_df.drop("Unnamed: 0", axis=1, inplace=True)

In [84]:
playercsv = "./Resources/player_data.csv"
player_df = pd.read_csv(playercsv)
merged_df = pd.merge(combined_df, player_df, left_on = "Player", right_on = "Name", how="left")
merged_df.drop(columns=["G", "GS", "Att", "Yds", "Att.1", "Yds.1", "Yds.2", "Name", "Salary"], inplace=True)
merged_df.drop("Unnamed: 0", axis=1, inplace=True)
merged_df.rename(columns={
    "Tm": "Team",
    "Pos": "Position",
    "Cmp": "Completions",
    "Int": "Interceptions",
    "Tgt": "Targets",
    "Y/R": "YdsPerRec",
    "Rec": "Receptions",
    "Year": "FantasyYear"
}, inplace=True)
merged_df.loc[merged_df["Player"]=="Christian McCaffrey"]

,Player,Team,Position,Age,Completions,Interceptions,Targets,Receptions,YdsPerRec,Fumbles,...,FantasyPoints,FantasyYear,PlayerID,Height,HeightFeet,HeightInches,Weight,Experience,Position,ADP
0,Christian McCaffrey,CAR,RB,23.0,0.0,0.0,142.0,116.0,8.66,1.0,...,469.2,2019,18877.0,"5'11""",5.0,11.0,205.0,5.0,RB,1.7
629,Christian McCaffrey,CAR,RB,22.0,1.0,0.0,124.0,107.0,8.10,4.0,...,385.5,2018,18877.0,"5'11""",5.0,11.0,205.0,5.0,RB,1.7
1302,Christian McCaffrey,CAR,RB,21.0,0.0,0.0,113.0,80.0,8.14,2.0,...,228.6,2017,18877.0,"5'11""",5.0,11.0,205.0,5.0,RB,1.7


In [85]:
merged_df.columns

Index(['Player', 'Team', 'Position', 'Age', 'Completions', 'Interceptions',
       'Targets', 'Receptions', 'YdsPerRec', 'Fumbles', 'FumblesLost',
       'PassingYds', 'PassingTD', 'PassingAtt', 'RushingYds', 'RushingTD',
       'RushingAtt', 'ReceivingYds', 'ReceivingTD', 'FantasyPoints',
       'FantasyYear', 'PlayerID', 'Height', 'HeightFeet', 'HeightInches',
       'Weight', 'Experience', 'Position', 'ADP'],
      dtype='object')